In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [4]:
# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [2]:
def create(act):
    baseMapNum = 32
    weight_decay = 1e-4

    model = Sequential()
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))
    
    return model

In [6]:
e_swish_2 = create(e_swish_2)

In [7]:
e_swish_2.load_weights("models/cifar10_normal_rms_ep250e_swish_2.h5")

In [9]:
probs_e_swish_2 = e_swish_2.predict_proba(x_test)

In [11]:
relu = create("relu")

In [13]:
relu.load_weights("models/cifar10_normal_rms_ep250relu.h5")

In [14]:
probs_relu = relu.predict_proba(x_test)

In [15]:
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)
swish = create(swish)
swish.load_weights("models/cifar10_normal_rms_ep250swish.h5")

In [16]:
probs_swish = swish.predict_proba(x_test)

In [17]:
del relu
del swish
del e_swish_2

In [24]:
elu = create("elu")
elu.load_weights("models/cifar10_normal_rms_ep250elu.h5")
probs_elu = elu.predict_proba(x_test)
del elu

In [25]:
def leaky_relu(x):
    return K.maximum(0.3*x, x)

leaky = create(leaky_relu)
leaky.load_weights("models/cifar10_normal_rms_ep250leaky_relu.h5")
probs_leaky = leaky.predict_proba(x_test)
del leaky

In [43]:
ensembled = probs_e_swish_2+probs_relu+probs_swish

In [44]:
y_hat = np.argmax(ensembled/3, axis=1)
y = np.argmax(y_test, axis=1)

good = np.sum(np.equal(y, y_hat))
print(float(good/len(y_test)))

0.9239


0.9239